In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from tensorflow.keras.preprocessing import image

import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
def predict_image(model, img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img) / 255.0 
    img_array = np.expand_dims(img_array, axis=0)  
    prediction = model.predict(img_array)[0][0]
    label = "Human" if prediction > 0.5 else "Animal"
    
    print(f"Prediction: {label} (confidence: {1 - prediction:.2f})" if label == "Human" else f"Prediction: {label} (confidence: {prediction:.2f})")
    return label

In [3]:
def binary_model(input_shape = (224,224,3)):
    model = Sequential([
        Conv2D(32, (5,5), padding='valid', activation='relu', input_shape=input_shape),
        MaxPooling2D(padding='valid'),
        
        Conv2D(64, (7,7), padding='valid', activation='relu', input_shape=input_shape),
        MaxPooling2D(padding='valid'),
        
        Conv2D(128, (7,7), padding='valid', activation='relu', input_shape=input_shape),
        MaxPooling2D(padding='valid'),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
        
    ])

    model.compile(
        optimizer = 'adam',
        loss = 'binary_crossentropy',
        metrics = ['accuracy']
    )

    return model

In [4]:
train_dir = 'binary_dataset_split/train/'
test_dir = 'binary_dataset_split/test/'

In [5]:
# Example way to check overlapping file names
train_files = set([f for f in os.listdir(train_dir + '/Human')] + [f for f in os.listdir(train_dir + '/Animal')])
test_files = set([f for f in os.listdir(test_dir + '/Human')] + [f for f in os.listdir(test_dir + '/Animal')])

overlap = train_files.intersection(test_files)
print("Overlap count:", len(overlap))

Overlap count: 0


In [6]:
early_stop = EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)
lr_decay = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=1, verbose=1)

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=.1,
    horizontal_flip=True
)

test_daragen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=10,
    class_mode='binary',
    shuffle=True
)

test_generator = test_daragen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=10,
    class_mode='binary',
    
)

Found 800 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [9]:
model = binary_model()

class_weight = {
    0: 1,        # Animal
    1: 1  # Humam
}

history = model.fit(
    train_generator,
    epochs = 5,
    validation_data = test_generator,
    class_weight=class_weight,
    callbacks=[early_stop, lr_decay]
)

Epoch 1/5
80/80 [==============================] - 161s 2s/step - loss: 0.5438 - accuracy: 0.9075 - val_loss: 0.0108 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/5
19/80 [======>.......................] - ETA: 1:51 - loss: 0.0080 - accuracy: 1.0000

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy')
plt.show()

In [ ]:
y_true = test_generator.classes  # Ground truth (0s and 1s)

# Predict on the test set
y_pred_probs = model.predict(test_generator)
y_pred = (y_pred_probs > 0.5).astype(int).reshape(-1)  # Convert probabilities to 0 or 1

# Classification Report (includes F1, Precision, Recall)
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Human', 'Animal']))

# Optional: Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

In [ ]:
vallist = ['animals10/raw-img/cane/OIP--oBsJIeAn_f5_DI1XJ9O6wHaF7.jpeg', 
           'animals10/raw-img/elefante/OIP-2OLFS8uisKPMfnJyGDcyWAHaFj.jpeg',
          'human/images/HipHop_HipHop1_C2_00810.png',
          'animals10/raw-img/pecora/OIP-4etnOV_GYvC-NqhILl1oYQHaFj.jpeg',
          'human/images/Jazz_Jazz2_C1_00405.png',
          'animals10/raw-img/gallina/112.jpeg']
for img in vallist:
    predict_image(model,img)